In [ ]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [2]:
sales_data = pd.read_csv("dataset/archive/201904 sales reciepts.csv")
sales_data.head()

,transaction_id,transaction_date,transaction_time,sales_outlet_id,staff_id,customer_id,instore_yn,order,line_item_id,product_id,quantity,line_item_amount,unit_price,promo_item_yn
0,7,2019-04-01,12:04:43,3,12,558,N,1,1,52,1,2.50,2.50,N
1,11,2019-04-01,15:54:39,3,17,781,N,1,1,27,2,7.00,3.50,N
2,19,2019-04-01,14:34:59,3,17,788,Y,1,1,46,2,5.00,2.50,N
3,32,2019-04-01,16:06:04,3,12,683,N,1,1,23,2,5.00,2.50,N
4,33,2019-04-01,19:18:37,3,17,99,Y,1,1,34,1,2.45,2.45,N


In [3]:
product_data = pd.read_csv("dataset/archive/product.csv")
product_data.head()

,product_id,product_group,product_category,product_type,product,product_description,unit_of_measure,current_wholesale_price,current_retail_price,tax_exempt_yn,promo_yn,new_product_yn
0,1,Whole Bean/Teas,Coffee beans,Organic Beans,Brazilian - Organic,It's like Carnival in a cup. Clean and smooth.,12 oz,14.40,$18.00,Y,N,N
1,2,Whole Bean/Teas,Coffee beans,House blend Beans,Our Old Time Diner Blend,Out packed blend of beans that is reminiscent ...,12 oz,14.40,$18.00,Y,N,N
2,3,Whole Bean/Teas,Coffee beans,Espresso Beans,Espresso Roast,Our house blend for a good espresso shot.,1 lb,11.80,$14.75,Y,N,N
3,4,Whole Bean/Teas,Coffee beans,Espresso Beans,Primo Espresso Roast,Our primium single source of hand roasted beans.,1 lb,16.36,$20.45,Y,N,N
4,5,Whole Bean/Teas,Coffee beans,Gourmet Beans,Columbian Medium Roast,A smooth cup of coffee any time of day.,1 lb,12.00,$15.00,Y,N,N


# Data Wrangling


In [4]:
dataset = pd.merge(sales_data[['transaction_id','transaction_date', 'sales_outlet_id', 'customer_id', 'product_id', 'quantity']],
                      product_data[['product_id', 'product_category', 'product']],
                      on ='product_id', how = 'left' )
dataset.head()

,transaction_id,transaction_date,sales_outlet_id,customer_id,product_id,quantity,product_category,product
0,7,2019-04-01,3,558,52,1,Tea,Traditional Blend Chai Rg
1,11,2019-04-01,3,781,27,2,Coffee,Brazilian Lg
2,19,2019-04-01,3,788,46,2,Tea,Serenity Green Tea Rg
3,32,2019-04-01,3,683,23,2,Coffee,Our Old Time Diner Blend Rg
4,33,2019-04-01,3,99,34,1,Coffee,Jamaican Coffee River Sm


# removes sizes


In [5]:
dataset[dataset["product"].str.contains("Dark chocolate")]['product'].unique()

array(['Dark chocolate Lg', 'Dark chocolate Rg', 'Dark chocolate'],
      dtype=object)

In [6]:
dataset['product'].nunique()

80

In [7]:
print(dataset['product'].unique())

['Traditional Blend Chai Rg' 'Brazilian Lg' 'Serenity Green Tea Rg'
 'Our Old Time Diner Blend Rg' 'Jamaican Coffee River Sm' 'Ethiopia Rg'
 'English Breakfast Lg' 'Sustainably Grown Organic Rg' 'Earl Grey Lg'
 'Jamaican Coffee River Rg' 'Serenity Green Tea Lg' 'Brazilian Sm'
 'English Breakfast Rg' 'Traditional Blend Chai Lg' 'Cappuccino'
 'Espresso shot' 'Cappuccino Lg' 'Latte' 'Earl Grey Rg'
 'Dark chocolate Lg' 'Columbian Medium Roast Sm' 'Oatmeal Scone'
 'Morning Sunrise Chai Lg' 'Morning Sunrise Chai Rg' 'Peppermint Lg'
 'Jumbo Savory Scone' 'Lemon Grass Lg' 'Sustainably Grown Organic Lg'
 'Dark chocolate Rg' 'Lemon Grass Rg' 'Ethiopia Sm' 'Latte Rg'
 'Our Old Time Diner Blend Sm' 'Chocolate Chip Biscotti'
 'Columbian Medium Roast Rg' 'Ethiopia Lg' 'Brazilian Rg'
 'Columbian Medium Roast Lg' 'Spicy Eye Opener Chai Rg' 'Ginger Biscotti'
 'Our Old Time Diner Blend Lg' 'Chocolate Croissant'
 'Jamaican Coffee River Lg' 'Hazelnut Biscotti' 'Spicy Eye Opener Chai Lg'
 'Cranberry Scone'

In [8]:
dataset['product']= dataset['product'].str.replace(' Rg','')
dataset['product']= dataset['product'].str.replace(' Sm','')
dataset['product']= dataset['product'].str.replace(' Lg','')

# product subset


In [9]:
products_to_take = ['Cappuccino', 'Latte', 'Espresso shot',  \
                     'Dark chocolate','Sugar Free Vanilla syrup', 'Chocolate syrup',\
                    'Carmel syrup', 'Hazelnut syrup', 'Ginger Scone',  \
                    'Chocolate Croissant', 'Jumbo Savory Scone', 'Cranberry Scone', 'Hazelnut Biscotti',\
                    'Croissant', 'Almond Croissant', 'Oatmeal Scone', 'Chocolate Chip Biscotti',\
                    'Ginger Biscotti',\
                   ]

In [10]:
dataset = dataset[dataset['product'].isin(products_to_take)]
dataset.head()

,transaction_id,transaction_date,sales_outlet_id,customer_id,product_id,quantity,product_category,product
16,108,2019-04-01,3,65,40,1,Coffee,Cappuccino
17,112,2019-04-01,3,90,37,2,Coffee,Espresso shot
20,127,2019-04-01,3,116,41,2,Coffee,Cappuccino
21,134,2019-04-01,3,189,38,2,Coffee,Latte
22,135,2019-04-01,3,131,40,1,Coffee,Cappuccino


In [11]:
dataset[['product','product_category']].drop_duplicates().reset_index(drop=True)

,product,product_category
0,Cappuccino,Coffee
1,Espresso shot,Coffee
2,Latte,Coffee
3,Dark chocolate,Drinking Chocolate
4,Oatmeal Scone,Bakery
5,Jumbo Savory Scone,Bakery
6,Chocolate Chip Biscotti,Bakery
7,Ginger Biscotti,Bakery
8,Chocolate Croissant,Bakery
9,Hazelnut Biscotti,Bakery


# clean transactions

In [12]:
dataset['transaction'] = dataset['transaction_id'].astype(str) +"_"+  dataset['customer_id'].astype(str)

In [13]:
dataset.head()

,transaction_id,transaction_date,sales_outlet_id,customer_id,product_id,quantity,product_category,product,transaction
16,108,2019-04-01,3,65,40,1,Coffee,Cappuccino,108_65
17,112,2019-04-01,3,90,37,2,Coffee,Espresso shot,112_90
20,127,2019-04-01,3,116,41,2,Coffee,Cappuccino,127_116
21,134,2019-04-01,3,189,38,2,Coffee,Latte,134_189
22,135,2019-04-01,3,131,40,1,Coffee,Cappuccino,135_131


In [14]:
num_of_items_for_each_transaction = dataset['transaction'].value_counts().reset_index()
valid_transactions = num_of_items_for_each_transaction[(num_of_items_for_each_transaction['count']>1)]['transaction'].tolist()
dataset = dataset[dataset['transaction'].isin(valid_transactions)].reset_index(drop=True)

# product trends

In [15]:
dataset['product_category'].value_counts()

product_category
Bakery                3800
Coffee                3174
Flavours              2246
Drinking Chocolate     947
Packaged Chocolate      22
Name: count, dtype: int64

# popularity 

In [16]:
popularity_recommendation = dataset.groupby(['product','product_category']).count().reset_index()
popularity_recommendation.head()

,product,product_category,transaction_id,transaction_date,sales_outlet_id,customer_id,product_id,quantity,transaction
0,Almond Croissant,Bakery,347,347,347,347,347,347,347
1,Cappuccino,Coffee,1290,1290,1290,1290,1290,1290,1290
2,Carmel syrup,Flavours,561,561,561,561,561,561,561
3,Chocolate Chip Biscotti,Bakery,352,352,352,352,352,352,352
4,Chocolate Croissant,Bakery,636,636,636,636,636,636,636


In [17]:
popularity_recommendation = popularity_recommendation[['product','product_category','transaction_id']]
popularity_recommendation = popularity_recommendation.rename(columns = {'transaction_id':'number_of_transactions'})

In [18]:
popularity_recommendation.to_csv('api/recommendation_objects/popularity_recommendation.csv', index=False)

# Apiori


In [19]:
train_basket = dataset.groupby(['transaction', 'product'])['product'].count().reset_index(name ='Count')

In [20]:
train_basket.head()

,transaction,product,Count
0,1000_0,Dark chocolate,1
1,1000_0,Oatmeal Scone,1
2,1001_8306,Cappuccino,1
3,1001_8306,Carmel syrup,1
4,1002_0,Carmel syrup,1


In [21]:
my_basket = train_basket.pivot_table(index='transaction', columns='product', values='Count').fillna(0)
my_basket.head()

product,Almond Croissant,Cappuccino,Carmel syrup,Chocolate Chip Biscotti,Chocolate Croissant,Chocolate syrup,Cranberry Scone,Croissant,Dark chocolate,Espresso shot,Ginger Biscotti,Ginger Scone,Hazelnut Biscotti,Hazelnut syrup,Jumbo Savory Scone,Latte,Oatmeal Scone,Sugar Free Vanilla syrup
transaction,,,,,,,,,,,,,,,,,,
1000_0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1001_8306,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1002_0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1004_5383,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1005_0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0


In [22]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1
    
basket_sets = my_basket.applymap(encode_units)
basket_sets.head()

C:\Users\Aniruddha Chaudhari\AppData\Local\Temp\ipykernel_12668\791096073.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  basket_sets = my_basket.applymap(encode_units)


product,Almond Croissant,Cappuccino,Carmel syrup,Chocolate Chip Biscotti,Chocolate Croissant,Chocolate syrup,Cranberry Scone,Croissant,Dark chocolate,Espresso shot,Ginger Biscotti,Ginger Scone,Hazelnut Biscotti,Hazelnut syrup,Jumbo Savory Scone,Latte,Oatmeal Scone,Sugar Free Vanilla syrup
transaction,,,,,,,,,,,,,,,,,,
1000_0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0
1001_8306,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1002_0,0,0,1,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0
1004_5383,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
1005_0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0


In [24]:
frequent_itemsets = apriori(basket_sets, min_support=0.05, use_colnames=True)
frequent_itemsets.head()

c:\Users\Aniruddha Chaudhari\AppData\Local\Programs\Python\Python310\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.115646,(Almond Croissant)
1,0.388889,(Cappuccino)
2,0.191232,(Carmel syrup)
3,0.112623,(Chocolate Chip Biscotti)
4,0.135676,(Chocolate Croissant)


In [32]:
rules_basket = association_rules(frequent_itemsets, metric="lift", min_threshold=1,num_itemsets=len(frequent_itemsets))

In [33]:
rules_basket.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(Cappuccino),(Almond Croissant),0.388889,0.115646,0.053288,0.137026,1.184874,1.0,0.008314,1.024775,0.255319,0.118090,0.024176,0.298905
1,(Almond Croissant),(Cappuccino),0.115646,0.388889,0.053288,0.460784,1.184874,1.0,0.008314,1.133333,0.176432,0.118090,0.117647,0.298905
2,(Dark chocolate),(Almond Croissant),0.277022,0.115646,0.057445,0.207367,1.793115,1.0,0.025409,1.115717,0.611791,0.171364,0.103715,0.352050
3,(Almond Croissant),(Dark chocolate),0.115646,0.277022,0.057445,0.496732,1.793115,1.0,0.025409,1.436567,0.500152,0.171364,0.303896,0.352050
4,(Latte),(Almond Croissant),0.382086,0.115646,0.054422,0.142433,1.231629,1.0,0.010235,1.031236,0.304358,0.122762,0.030290,0.306511


In [35]:
rules_basket[rules_basket['antecedents']=={'Latte'}].sort_values(by='confidence', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
70,(Latte),(Sugar Free Vanilla syrup),0.382086,0.200302,0.108844,0.284866,1.422182,1.0,0.032311,1.118249,0.480415,0.229848,0.105745,0.414131
32,(Latte),(Carmel syrup),0.382086,0.191232,0.107710,0.281899,1.474121,1.0,0.034643,1.126260,0.520509,0.231331,0.112105,0.422570
44,(Latte),(Chocolate syrup),0.382086,0.188964,0.103175,0.270030,1.428997,1.0,0.030974,1.111053,0.485842,0.220517,0.099953,0.408015
68,(Latte),(Hazelnut syrup),0.382086,0.171580,0.101285,0.265084,1.544961,1.0,0.035727,1.127231,0.570848,0.223893,0.112871,0.427696
52,(Latte),(Croissant),0.382086,0.114135,0.057067,0.149357,1.308605,1.0,0.013458,1.041407,0.381651,0.129948,0.039761,0.324679
38,(Latte),(Chocolate Croissant),0.382086,0.135676,0.055178,0.144411,1.064381,1.0,0.003338,1.010209,0.097889,0.119281,0.010106,0.275548
66,(Latte),(Ginger Scone),0.382086,0.133409,0.055178,0.144411,1.082472,1.0,0.004204,1.012860,0.123300,0.119869,0.012696,0.279005
4,(Latte),(Almond Croissant),0.382086,0.115646,0.054422,0.142433,1.231629,1.0,0.010235,1.031236,0.304358,0.122762,0.030290,0.306511
64,(Latte),(Ginger Biscotti),0.382086,0.106198,0.054044,0.141444,1.331890,1.0,0.013467,1.041053,0.403272,0.124456,0.039434,0.325170
48,(Latte),(Cranberry Scone),0.382086,0.116024,0.051398,0.134520,1.159416,1.0,0.007067,1.021371,0.222518,0.115059,0.020924,0.288759


# Save in Json


In [43]:
product_categories = dataset[['product','product_category']].drop_duplicates().set_index('product').to_dict()['product_category']

In [44]:
product_categories

{'Cappuccino': 'Coffee',
 'Jumbo Savory Scone': 'Bakery',
 'Latte': 'Coffee',
 'Chocolate Chip Biscotti': 'Bakery',
 'Espresso shot': 'Coffee',
 'Hazelnut Biscotti': 'Bakery',
 'Chocolate Croissant': 'Bakery',
 'Dark chocolate': 'Packaged Chocolate',
 'Cranberry Scone': 'Bakery',
 'Croissant': 'Bakery',
 'Almond Croissant': 'Bakery',
 'Ginger Biscotti': 'Bakery',
 'Oatmeal Scone': 'Bakery',
 'Ginger Scone': 'Bakery',
 'Chocolate syrup': 'Flavours',
 'Hazelnut syrup': 'Flavours',
 'Carmel syrup': 'Flavours',
 'Sugar Free Vanilla syrup': 'Flavours'}

In [50]:
recommendations_json = {}

antecedents = rules_basket['antecedents'].unique()
for antecedent in antecedents:
    df_rec = rules_basket[rules_basket['antecedents']==antecedent]
    df_rec = df_rec.sort_values('confidence',ascending=False)
    key = "_".join(antecedent)
    recommendations_json[key] = []
    for _, row in df_rec.iterrows():
        rec_objects =row['consequents']
        for rec_object in rec_objects:
            already_exists = False
            for current_rec_object in recommendations_json[key]:
                if rec_object == current_rec_object['product']:
                    already_exists=True
            if already_exists:
                continue
            
            rec = {'product':rec_object, 
                   'product_category':product_categories[rec_object],
                   'confidence': row['confidence']
                  }
            recommendations_json[key].append(rec)

In [51]:
import pprint
pprint.pp(recommendation_json)

{'Cappuccino': [{'product': 'Sugar Free Vanilla syrup',
                 'product_category': 'Flavours',
                 'confidence': 0.29057337220602525},
                {'product': 'Chocolate syrup',
                 'product_category': 'Flavours',
                 'confidence': 0.2818270165208941},
                {'product': 'Carmel syrup',
                 'product_category': 'Flavours',
                 'confidence': 0.26433430515063167},
                {'product': 'Hazelnut syrup',
                 'product_category': 'Flavours',
                 'confidence': 0.23226433430515062},
                {'product': 'Ginger Scone',
                 'product_category': 'Bakery',
                 'confidence': 0.15354713313896987},
                {'product': 'Chocolate Chip Biscotti',
                 'product_category': 'Bakery',
                 'confidence': 0.14480077745383868},
                {'product': 'Chocolate Croissant',
                 'product_category': 'Bakery',
   

In [52]:
import json
with open('api/recommendation_objects/recommendations.json', 'w') as f:
    json.dump(recommendations_json, f)